In [68]:
try:
    import  pyyaml
except:
  !pip install pyyaml==5.1
from pycocotools.coco import COCO
import os
from PIL import Image
import skimage.io as io
from pylabel import importer

import os
from pathlib import Path
import pandas as pd
import json
import time
import datetime

import numpy as np
import matplotlib.pyplot as plt
import os, json, cv2, random

# from google.colab.patches import cv2_imshow

import matplotlib.patches as patches
import matplotlib
import matplotlib.colors as mcolors
plt.rcParams.update({'font.size': 22})
style_label = 'fivethirtyeight' #plt.style.available 
try:
    import dsatools 
except:
    !pip install dsatools
finally:
    import dsatools.utilits as ut
from datetime import datetime
import csv

In [2]:
def coco_datasets_info(list_images, list_anno):

    datasets_list = list()

    for i in range(len(list_anno)):

        if os.path.exists(list_anno[i]) and os.path.exists(list_images[i]): 


            dict_json = load_coco_json(json_file = list_anno[i], 
                                image_root = list_images[i], 
                                dataset_name = 'None',
                                extra_annotation_keys = None)
            dataset_info_dict = dict()      
            
            # dict_json = corret_coco_json_path(dict_json, list_images[i])

            name_dataset = os.path.split(os.path.split(os.path.split(list_anno[i])[0])[0])[1]

            dataset_info_dict['name']   = name_dataset
            dataset_info_dict['length'] = len(dict_json)

            df = pd.DataFrame(dict_json)

            dataset_info_dict['anno_path'] = list_anno[i]
            dataset_info_dict['image_path'] = list_images[i]

            dataset_info_dict['height'] = list(set(df['height']))
            dataset_info_dict['width']  = list(set(df['width']))

            coco=COCO(list_anno[i], )

            dataset_info_dict['anno_number'] = len(coco.anns)

            cats = coco.loadCats(coco.getCatIds())

            class_id = [cat['id'] for cat in cats]
            class_names = [cat['name'] for cat in cats]

            dataset_info_dict['class_id']  = class_id
            dataset_info_dict['class_names']  = class_names

            dataset_info_dict['COCO_obj']  = coco

            datasets_list +=[dataset_info_dict]
    return datasets_list

In [3]:
path = os.getcwd()
path = path[:path.find('src')]
path_2_project = os.path.join(path,'PitStonesData')
print(path_2_project)
list_project_dir =os.listdir(path_2_project)
list_project_dir

/home/kirill/Учеба/asbestos/PitStonesData


['data_GRAINet_demo',
 'open-pit-zoomed-161120',
 'GRAINet_demo_data.zip',
 'pits_stones_detections_300920',
 'pits_stones_detections_161120',
 'pits_stones_detections_020221',
 'test transport',
 'asbestos_stone_lab_common_camera',
 'pits_stones_detections_111121',
 'pits_stones_detections_161220',
 'asbestos_stone_020221',
 'asbest_stones_111121',
 'asbestos_stone_161220']

In [112]:
list_images = [os.path.join(path_2_project,data_pth,'images') for data_pth in list_project_dir]
list_anno   = [os.path.join(path_2_project,data_pth,'annotations','instances_default.json') for data_pth in list_project_dir]
list_images, list_anno

(['/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_020221/images',
  '/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_300920/images',
  '/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_161120/images',
  '/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_111121/images',
  '/home/kirill/Учеба/asbestos/PitStonesData/asbestos_stone_lab_common_camera/images',
  '/home/kirill/Учеба/asbestos/PitStonesData/drive-download-20220131T131533Z-001.zip/images',
  '/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_161220/images',
  '/home/kirill/Учеба/asbestos/PitStonesData/coco_eval/images',
  '/home/kirill/Учеба/asbestos/PitStonesData/stones_transport/images'],
 ['/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_020221/annotations/instances_default.json',
  '/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_300920/annotations/instances_default.json',
  '/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_161120/annotations/instances_default.json',
  '/home/kirill

In [113]:
import torch

TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION  = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION, '\n', 10*'-')

try:
    import detectron2
except:
  # Install detectron2 that matches the above pytorch version
  # See https://detectron2.readthedocs.io/tutorials/install.html for instructions
    !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html

  # If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.
    exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime
    
from detectron2.data.datasets    import register_coco_instances, load_coco_json

torch:  1.10 ; cuda:  cu102 
 ----------


In [114]:
coco_datasets_list = coco_datasets_info(list_images, list_anno);

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.12s)
creating index...
index created!
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
loading annotations into memory...
Done (t=0.13s)
creating index...
index created!
loading annotations into memory...
Done (t=0.09s)
creating index...
index created!


In [260]:
df= pd.DataFrame(coco_datasets_list)
df.sum()

name           pits_stones_020221pits_stones_300920pits_stone...
length                                                       311
anno_path      /home/kirill/Учеба/asbestos/PitStonesData/pits...
image_path     /home/kirill/Учеба/asbestos/PitStonesData/pits...
height          [2048, 2048, 2048, 2048, 3456, 5184, 2048, 2048]
width           [2592, 2592, 2592, 2592, 5184, 3456, 2592, 2592]
anno_number                                                12926
class_id              [1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2]
class_names    [stone, asbest, stone, asbest, stone, asbest, ...
dtype: object

In [280]:
import csv 

def get_time_date(s):
    l = s.split('.')
    if len(l) == 2:
        s=l[0]
    elif len(l) == 3:
        s=l[1]
    s = s.split('/')[-1]
    if len(s.split('-')) == 6:
        return datetime.strptime(s.split('_')[1], '%d-%m-%Y-%H-%M-%S')
    elif len(s.split('_')) == 4:
        if len(s.split(':')) == 3:
            l = s.split('_')
            s = l[1]+'_' + l[2]
            return datetime.strptime(s, '%H:%M:%S_%d-%m-%Y')  
        else:
            return datetime.strptime(s.split('Image_')[1], '%H_%M_%S')      
i = 1
with open('image_times.csv','w',newline='') as csvfile:
    for coco_dataset in coco_datasets_list:
        for c in coco_dataset['COCO_obj'].dataset['images']:
            date = get_time_date(c['file_name'])
            if date:
                d = {'dataset':coco_dataset['name'], 'name':c['file_name'].split('/')[-1], 'time': str(date.time()), 'date': str(date.date())}
            else:
                d = {'dataset':coco_dataset['name'], 'name':c['file_name'].split('/')[-1], 'time': None, 'date': None}
            writer = csv.writer(csvfile, delimiter=';',quoting=csv.QUOTE_MINIMAL)
            writer.writerow(d.values())
            i+=1

In [278]:
writer

In [124]:
for coco_set in coco_datasets_list:
    image_names = os.listdir(coco_set['image_path'])
    print(coco_set['image_path'])
    for name in image_names:
        print(name)

/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_020221/images
asbestos
/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_300920/images
asbestos
/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_161120/images
asbestos
/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_111121/images
ManualGeneralImage_13_58_58.bmp
GeneralImage_11-11-2021-13-34-14.bmp
ManualGeneralImage_14_01_51.bmp
ManualGeneralImage_13_55_25.bmp
GeneralImage_11-11-2021-13-43-18.bmp
ManualGeneralImage_13_59_48.bmp
ManualGeneralImage_13_56_14.bmp
ManualGeneralImage_13_45_07.bmp
ManualGeneralImage_14_00_56.bmp
GeneralImage_11-11-2021-12-24-03.bmp
ManualGeneralImage_12_54_52.bmp
ManualGeneralImage_12_54_15.bmp
ManualGeneralImage_13_36_55.bmp
GeneralImage_11-11-2021-13-27-54.bmp
GeneralImage_11-11-2021-13-24-49.bmp
ManualGeneralImage_13_37_47.bmp
ManualGeneralImage_12_55_46.bmp
ManualGeneralImage_13_53_22.bmp
ManualGeneralImage_13_38_41.bmp
/home/kirill/Учеба/asbestos/PitStonesData/asbestos_stone_lab_common_

In [6]:
!pwd

/home/kirill/Учеба/asbestos/src/notebooks


### Convert coco to yolo

In [90]:
from pathlib import Path

In [108]:
path_to_annotations = "/home/kirill/Учеба/asbestos/PitStonesData/open-pit-zoomed-161120/annotations/instances_default.json"
path_to_images = "/home/kirill/Учеба/asbestos/PitStonesData/open-pit-zoomed-161120/images/asbestos/pits/"
dataset = importer.ImportCoco(path_to_annotations, path_to_images=path_to_images, name="pits_stones_detections_300920")

In [109]:
print(f"Number of images: {dataset.analyze.num_images}")
print(f"Number of classes: {dataset.analyze.num_classes}")
print(f"Classes:{dataset.analyze.classes}")
print(f"Class counts:\n{dataset.analyze.class_counts}")
print(f"Path to annotations:\n{dataset.path_to_annotations}")

Number of images: 24
Number of classes: 1
Classes:['stone']
Class counts:
stone    218
Name: cat_name, dtype: int64
Path to annotations:
/home/kirill/Учеба/asbestos/PitStonesData/open-pit-zoomed-161120/annotations


In [110]:
dataset.df

,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,...,ann_segmentation,ann_iscrowd,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split,annotated
id,,,,,,,,,,,,,,,,,,,,,
0,/home/kirill/Учеба/asbestos/PitStonesData/open...,zoomed_ManualGeneralImage_09_27_13.bmp,,1,2592,2048,,,149.19,820.41,...,"[[943.61, 1644.87, 915.81, 1664.4, 898.53, 167...",0,,,,1,stone,,,1
1,/home/kirill/Учеба/asbestos/PitStonesData/open...,zoomed_ManualGeneralImage_09_27_13.bmp,,1,2592,2048,,,1125.40,1254.99,...,"[[1683.56, 1825.92, 1691.82, 1824.04, 1700.84,...",0,,,,1,stone,,,1
2,/home/kirill/Учеба/asbestos/PitStonesData/open...,zoomed_ManualGeneralImage_09_27_13.bmp,,1,2592,2048,,,1542.33,934.97,...,"[[2126.4, 1097.61, 2131.29, 1082.21, 2144.43, ...",0,,,,1,stone,,,1
3,/home/kirill/Учеба/asbestos/PitStonesData/open...,zoomed_ManualGeneralImage_09_27_13.bmp,,1,2592,2048,,,929.30,266.89,...,"[[1429.41, 1132.17, 1459.4, 1100.41, 1493.8, 1...",0,,,,1,stone,,,1
4,/home/kirill/Учеба/asbestos/PitStonesData/open...,zoomed_ManualGeneralImage_09_27_13.bmp,,1,2592,2048,,,2099.02,100.48,...,"[[2590.02, 845.8, 2572.38, 827.42, 2545.92, 81...",0,,,,1,stone,,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,/home/kirill/Учеба/asbestos/PitStonesData/open...,zoomed_Full_637437067053998640.bmp,,24,2592,2048,,,1855.87,622.14,...,"[[2097.31, 953.93, 2103.43, 917.69, 2129.47, 8...",0,,,,1,stone,,,1
214,/home/kirill/Учеба/asbestos/PitStonesData/open...,zoomed_Full_637437067053998640.bmp,,24,2592,2048,,,2295.36,1314.04,...,"[[2316.2, 1327.23, 2354.06, 1325.95, 2381.28, ...",0,,,,1,stone,,,1
215,/home/kirill/Учеба/asbestos/PitStonesData/open...,zoomed_Full_637437067053998640.bmp,,24,2592,2048,,,1703.22,1174.84,...,"[[1817.71, 1331.87, 1835.43, 1325.14, 1847.49,...",0,,,,1,stone,,,1


In [111]:
dataset.export.ExportToYoloV5("/home/kirill/Учеба/asbestos/PitStonesData/yolo_annotation/", cat_id_index=0)[0]

'/home/kirill/Учеба/asbestos/PitStonesData/dataset.yaml'

In [112]:
data_paths = [
"/home/kirill/Учеба/asbestos/PitStonesData/asbestos_stone_020221",
"/home/kirill/Учеба/asbestos/PitStonesData/asbestos_stone_161220",
"/home/kirill/Учеба/asbestos/PitStonesData/asbest_stones_111121",
"/home/kirill/Учеба/asbestos/PitStonesData/open-pit-zoomed-161120",
"/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_detections_020221",
"/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_detections_111121",
"/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_detections_161120",
"/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_detections_161220",
"/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_detections_300920",
"/home/kirill/Учеба/asbestos/PitStonesData/test transport"
]
list_path_to_images = []
list_path_to_annotations = []
for p in data_paths:
    d = p+"/images"
    list_path_to_annotations.append(p + "/annotations/instances_default.json")
    count_files = 1
    while count_files<2:
        dirs = os.listdir(d)
        count_files = len(dirs)
        if count_files > 1:
            list_path_to_images.append(d)
            break
        d = os.path.join(d, dirs[0])

In [122]:
# ### rename
# path = "/home/kirill/Учеба/asbestos/PitStonesData/open-pit-zoomed-161120/images/"
# files = os.listdir(path)
# for file in files:
#     os.rename(path+file, path+"zoomed_"+file.replace('zoomed_zoomed_',''))

In [114]:
yolo_dir = "yolo_annotation"
path_2_yolo = "/home/kirill/Учеба/asbestos/PitStonesData/yolo_annotation"
for path_2_anno, path_2_image, path_2_data in zip(list_path_to_annotations,list_path_to_images, data_paths):
    name_set = path_2_data.split('/')[-1]
    dataset = importer.ImportCoco(path_2_anno,path_2_image, name_set )
    index = dataset.df.img_folder[0].find("Data/")
    head = dataset.df.img_folder[0][:index + 5]
    tail = dataset.df.img_folder[0][index + 5:].replace("images/",'').replace(name_set,'')                                                                                                 
    path_2_save = os.path.join(head+yolo_dir+tail)
    print(path_2_save, tail)
    try:
        os.makedirs(path_2_save)
    except:
        print('Do not create dir: ', path_2_save)
    dataset.export.ExportToYoloV5(output_path=path_2_yolo, cat_id_index=0)

/home/kirill/Учеба/asbestos/PitStonesData/yolo_annotation/asbestos/stones/020221 /asbestos/stones/020221
/home/kirill/Учеба/asbestos/PitStonesData/yolo_annotation/asbestos/stones/161220 /asbestos/stones/161220
/home/kirill/Учеба/asbestos/PitStonesData/yolo_annotation/images /images
/home/kirill/Учеба/asbestos/PitStonesData/yolo_annotation/images /images
Do not create dir:  /home/kirill/Учеба/asbestos/PitStonesData/yolo_annotation/images
/home/kirill/Учеба/asbestos/PitStonesData/yolo_annotation/asbestos/pits/020221 /asbestos/pits/020221
/home/kirill/Учеба/asbestos/PitStonesData/yolo_annotation/images /images
Do not create dir:  /home/kirill/Учеба/asbestos/PitStonesData/yolo_annotation/images
/home/kirill/Учеба/asbestos/PitStonesData/yolo_annotation/asbestos/pits/161120 /asbestos/pits/161120
/home/kirill/Учеба/asbestos/PitStonesData/yolo_annotation/asbestos/pits/161220 /asbestos/pits/161220
/home/kirill/Учеба/asbestos/PitStonesData/yolo_annotation/asbestos/pits/300920 /asbestos/pits/3009

In [115]:
### delete asbest segmentation

In [117]:
yolo_files = os.listdir(path_2_yolo)
yolo_files.remove('stone')
for name in yolo_files:
    outfile = open(os.path.join(path_2_yolo,"stone", name), "w")
    file = open(os.path.join(path_2_yolo,name), 'r') 
    lines = file.readlines()
    for line in lines:
        if line in lines:
            if line[0] == "2":
                continue
            else:
                outfile.write(line)
    outfile.close()
    file.close()

In [119]:
with open('open_pit_images.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=';',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(['dataset','image'])
    for path_2_anno, path_2_image, path_2_data in zip(list_path_to_annotations,list_path_to_images, data_paths):
        name_set = path_2_data.split('/')[-1]
        list_images = os.listdir(path_2_image)
        for image_name in list_images:
            spamwriter.writerow([name_set,image_name])